In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install med2image
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import glob
import re
import time
import sys

In [3]:
def sort_list(l):
    convert = lambda text: int(text) if text.isdigit() else text
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)', key)]
    return sorted(l, key = alphanum_key)

In [4]:
def create_image(dirname, label):
    if dirname[-1] == "/":
        dirname = dirname[:-1]
    images = glob.glob(str(dirname) + "/*.png")

    # Sort images alphanumerically by filename to ensure that z loops from front to back
    images = sort_list(images)

    p = np.zeros((len(images), Image.open(images[0]).convert('L').size[0]))

    # setup toolbar
    toolbar_width = int(len(images)/5 + 1)
    sys.stdout.write(label + " [%s]" % (" " * toolbar_width))
    sys.stdout.flush()
    sys.stdout.write("\b" * (toolbar_width+1))

    # Loop through CT slices from front to back
    for z in range(len(images)):
        img = Image.open(images[z]).convert('L')  # convert image to 8-bit grayscale
        HEIGHT, WIDTH = img.size
        data = list(img.getdata()) # convert image data to a list of integers
        # convert that to 2D list (list of lists of integers)
        pixels = [data[offset:offset+WIDTH] for offset in range(0, WIDTH*HEIGHT, WIDTH)]

        # Loop from left to right on the CT slice
        for x in range(WIDTH):
            # Sum y values in the current x column
            sum = 0
            for y in range(HEIGHT):
                sum += pixels[y][x]
            # Assign sum to the point (x, z) on the coronal image - p[z][x] in the pixel array, 
            # since z represents height (rows) and x represents length (columns)
            p[len(images) - 1 - z][x] = sum

        if z % 5 == 0:
            # update the bar
            sys.stdout.write("-")
            sys.stdout.flush()
    sys.stdout.write("]\n")
    
    return p

def save(p, filename):
    plt.clf()
    plt.imshow(p, cmap='gray')
    plt.gca().set_axis_off()
    plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, 
                hspace = 0, wspace = 0)
    plt.margins(0,0)
    plt.gca().xaxis.set_major_locator(plt.NullLocator())
    plt.gca().yaxis.set_major_locator(plt.NullLocator())
    plt.savefig(filename, bbox_inches = 'tight',
        pad_inches = 0)
    
    final_img = Image.open(filename)
    size = 300
    if final_img.size[1] < 300:
        final_img = final_img.resize((final_img.size[0], 300))    

    final_img.save(filename)

In [5]:
%cd /content/drive/My Drive/

/content/drive/My Drive


In [6]:
ret = []

def floodfill(x, y):
    if matrix[x][y] != 0: 
        matrix[x][y] = 0 
        ret.append((x,y))
        if x > 0:
            floodfill(x-1,y)
        if x < matrix.shape[0] - 1:
            floodfill(x+1,y)
        if y > 0:
            floodfill(x,y-1)
        if y < matrix.shape[1] - 1:
            floodfill(x,y+1)

def flood():
    arr = []
    for r in range(matrix.shape[0]):
        for c in range(matrix.shape[1]):
            if (matrix[r][c] != 0):
                ret.clear()
                floodfill(r, c)
                m = []
                for i in ret:
                    m.append(i)
                arr.append(m)
    return arr

In [11]:
from google.colab import files
import json
import random

sys.setrecursionlimit(100000)
data = {}

for i in range(45):
    mask_name = './data/masks/Volume' + str(i)
    matrix = create_image(mask_name, 'Creating mask ' + str(i) + ':')
    arr = flood()
    size = random.randint(10000,99999)
    fname = 'xray' + str(i) + '.jpg'
    name = fname + str(size)
    
    regions = []
    for a in arr:
        minX = 10000
        maxX = 0
        minY = 10000
        maxY = 0
        for tup in a:
            if tup[0] > maxX:
                maxX = tup[0]
            if tup[0] < minX:
                minX = tup[0]
            if tup[1] > maxY:
                maxY = tup[1]
            if tup[1] < minY:
                minY = tup[1]
        all_points_x = [minX, minX, maxX, maxX]
        all_points_y = [minY, maxY, maxY, minY]
        
        shape_attributes = {}
        shape_attributes['name'] = 'polygon'
        shape_attributes['all_points_x'] = all_points_y
        shape_attributes['all_points_y'] = all_points_x

        image_quality = {}
        image_quality['good'] = True
        image_quality['frontal'] = True
        image_quality['good_illumination'] = True

        region_attributes = {}
        region_attributes['name'] = 'not_defined'
        region_attributes['type'] = 'unknown'
        region_attributes['image_quality'] = image_quality
        
        regions.append({
            'shape_attributes':shape_attributes,
            "region_attributes": region_attributes
        })

    file_attributes = {}
    file_attributes['caption'] = ''
    file_attributes['public_domain'] = 'no'
    file_attributes['image_url'] = ''

    data[name] = {}
    data[name]['filename'] = fname
    data[name]['size'] = size
    data[name]['regions'] = regions
    data[name]['file_attributes'] = file_attributes

with open('via_region_data.json', 'w') as outfile:
    json.dump(data, outfile)
files.download('via_region_data.json')

Creating mask 31: [                                                             ]-------------------------------------------------------------]
Creating mask 32: [                                         ]----------------------------------------]
Creating mask 33: [                                         ]----------------------------------------]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>